В задании вам понадобится собрать генеративную модлель для языка

---

# LSTM (7 баллов)

В данной части нужно реализовать модель с ипользованием LSTM

In [1]:
!pip install --quiet sentencepiece datasets transformers


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
from torch.utils.data import DataLoader
from dataset_maker import DatasetMaker, collate_batch
from data_generator import DataGenerator

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

Ниже вам нужно реализовать модель, которая по началу последовательности предсказывает следующий токен.
*   Модель получает на вход последовательность токенов, прогоняет её через LSTM и выдает вероятности следующего токена.  
*   Используйте LSTM из pytorch
*   Не забудьте про `batch_first`

In [4]:
DIRECTORY = './data/books_txt'  
VOCAB_PATH = './data/vocab.txt'
MAX_SEQ_LEN = 256
BATCH_SIZE = 16
OFFSET = MAX_SEQ_LEN // 2
TEST_SIZE = 0.03

In [5]:
dataset_maker = DatasetMaker(directory=DIRECTORY, test_size=TEST_SIZE, vocab_path=VOCAB_PATH, max_seq_len=MAX_SEQ_LEN, batch_size=BATCH_SIZE, offset=OFFSET, create_test_segments=True)
train_dataloader = DataLoader(dataset_maker, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(dataset_maker.segments_test, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

In [6]:
dataset_maker.tokenizer.vocab_size

8000

In [7]:
LEARNING_RATE = 0.0005
EMBEDDING_DIM = 512
NUM_LAYERS = 2
PAD_TOKEN_IDX = 0
SAVE_PATH = './saved_models'

In [8]:
model = DataGenerator(vocab_size=dataset_maker.tokenizer.vocab_size, test_dataloader=test_dataloader, embed_dim=EMBEDDING_DIM, num_layers=NUM_LAYERS, pad_token_idx=PAD_TOKEN_IDX, device=device)

### Реализуйте обучение модели:
*  Не забудьте сдвинуть src и trg относительно друг друга.
*  Не забудьте про `clip_grad_norm_`
*  Данных очень много, для отладки лучше проходить только часть данных иначе этоха будет очень длинной

Получите `loss < 5.0` на тестовой выборке. 

Если модель обучается слишком быстро до значений <1.0 вы что-то напутали с данными.

In [ ]:
model.train_loop(epochs=100, train_dataloader=train_dataloader, lr=LEARNING_RATE, save_path=SAVE_PATH)

Epoch 1/100


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 61%|██████    | 10335/17049 [2:20:02<1:29:19,  1.25it/s]

# Генерация текста (5 баллов) 

Реализуйте функцию, которая продолжает текст.
1.   Переведите строчку в токены
2.   Реализуйте код который предсказывает вероятность следующей буквы
3.   Семплируйте следующую букву
4.   Повторяйте пункты 2-3 в цикле
5.   Преобразуйте токены в строчку

In [9]:
model.load_model(SAVE_PATH)

Loaded model from model_epoch_50.pth


/Users/ivanzolin/Documents/itmo-magistracy/dl-practice/data_generator.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=

In [10]:
model.continues_sentence("Я помню чудное мгновенье", dataset_maker.tokenizer)

'я помню чудное мгновенье, когда я был в восторге, когда я был счастлив, и я, как будто я, не мог не видеть, что я не могу быть'

In [11]:
model.continues_sentence("Мой дядя самых честных правил,", dataset_maker.tokenizer)

'мои дядя самых честных правил, и я, как и я, не знаю, что я такое. я не знаю, что я говорю, но я не могу не верить, что'

In [14]:
model.continues_sentence("Четыре года потратил Деонардо на", dataset_maker.tokenizer)

'четыре года потратил деонардо на себя. - - - - - - - - - - - - - - - - - - - - - - - - - - - -'

In [13]:
model.continues_sentence("Если сила плохих людей в том, что они вместе, то хорошим людям, чтобы стать силой, надо", dataset_maker.tokenizer)

'если сила плохих людеи в том, что они вместе, то хорошим людям, чтобы стать силои, надо было бы, чтобы они были так же счастливы, как и они, и потому не могли бы быть счастливыми. и потому, что они не'